## Constants

In [11]:
import os

file_count = 50   # TODO: change to 1000
playlists_per_file = 1000

dir_data_raw = os.path.join("..", "data", "raw")
dir_data_processed = os.path.join("..", "data", "processed")

playlist_count = file_count * playlists_per_file

Merge the data extracted from the Spotify API (csv file) with that of the Spotify Million Playlist Dataset (pickle file)

In [41]:
import os
import pandas as pd

path_audio_features = os.path.join(dir_data_raw, "tracks", "track_audio_features.csv")
audio_features = pd.read_csv(path_audio_features)

display(audio_features)

,id,tempo,key,mode,loudness,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,duration_ms,time_signature
0,3zyYVItCMCjFzBHTyjrxPK,79.227,8.0,1.0,-3.406,0.466,0.856,0.3180,0.257000,0.000000,0.6750,0.531,292223.0,4.0
1,4jSy0HTIoC9yiwZ8OVyTCW,164.207,1.0,1.0,-7.972,0.510,0.849,0.1190,0.000546,0.000748,0.6890,0.887,315067.0,4.0
2,4zyqBSUFNkJ20mw1FB68gt,83.947,4.0,0.0,-22.867,0.308,0.114,0.0321,0.958000,0.902000,0.0853,0.303,350906.0,4.0
3,63B3TtwUzOoJoe3unMteVa,93.696,1.0,0.0,-4.166,0.660,0.943,0.2770,0.129000,0.000000,0.5570,0.599,210733.0,4.0
4,7y9iMe8SOB6z3NoHE2OfXl,118.384,0.0,1.0,-3.539,0.675,0.751,0.0296,0.060400,0.000000,0.0893,0.612,181279.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38144,0s9iofqbQdsmWZQJ1TZh1c,118.061,7.0,1.0,-13.766,0.550,0.351,0.0443,0.531000,0.006020,0.1500,0.237,162826.0,4.0
38145,5Ozvg38CnCQKmPP8tgSRA6,96.520,0.0,1.0,-4.130,0.609,0.742,0.0477,0.001310,0.000002,0.0725,0.531,256227.0,4.0
38146,2yWsZpLZNmap92YkGkzyDq,179.985,6.0,1.0,-5.899,0.373,0.584,0.0272,0.004810,0.449000,0.1630,0.403,214667.0,3.0
38147,7JxA6bPIyXNohnrTw6CzS0,178.012,7.0,0.0,-7.096,0.473,0.829,0.3620,0.048900,0.000000,0.2530,0.583,270680.0,4.0


Create a function to compute relevant metrics about a playlist's audio features.

In [163]:
def get_playlist_metrics(df: pd.DataFrame):
    def get_statistic(f):
        target_columns = list(range(2,9)) # TODO: which columns do we care about?
        return df.iloc[:, target_columns].apply(f).to_frame().transpose()

    return pd.concat({"Mean" : get_statistic(pd.Series.mean),
                      "Median" : get_statistic(pd.Series.median),
                      "Standard deviation" : get_statistic(pd.Series.std),
                      "Variance" : get_statistic(pd.Series.var)}, axis=1)
    

Combine extracted and calculated data into a playlist frame

In [164]:
import pickle

playlist_list_path = os.path.join(dir_data_processed, "playlists" + str(playlist_count) + ".pkl")

with open(playlist_list_path, "rb") as fin:
    playlist_data = pickle.load(fin)

for playlist in playlist_data[2:3]:
    print("Number of tracks: {}".format(len(playlist["track_ids"])))    
    playlist_features = pd.DataFrame(columns = audio_features.columns.tolist())

    for track_id in playlist["track_ids"]:
        track_features = audio_features[audio_features["id"] == track_id]
        
        if track_features.empty: # TODO: handle cases when track id has no audio features
            # print("{} has no audio features".format(track_id))
            continue

        playlist_features = pd.concat([playlist_features, track_features])
    display(playlist_features)
    display(get_playlist_metrics(playlist_features))

Number of tracks: 72


/var/folders/b6/ns9gsf2x6lq54_53qrbw8v440000gn/T/ipykernel_3723/263009295.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  playlist_features = pd.concat([playlist_features, track_features])


,id,tempo,key,mode,loudness,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,duration_ms,time_signature
73,02opp1cycqiFNDpLd2o1J3,170.861,6.0,0.0,-4.888,0.442,0.684,0.1680,0.1910,0.000000,0.0725,0.218,207680.0,4.0
6277,08zJpaUQVi9FrKv2e32Bah,129.336,6.0,1.0,-7.738,0.688,0.556,0.2640,0.7000,0.000002,0.1080,0.416,240320.0,4.0
16808,4NTWZqvfQTlOMitlVn6tew,143.067,7.0,1.0,-3.839,0.591,0.889,0.1150,0.0189,0.000000,0.1550,0.650,239613.0,4.0
1849,6gBFPUFcJLzWGx4lenP6h2,130.049,7.0,1.0,-3.370,0.841,0.728,0.0484,0.0847,0.000000,0.1490,0.430,243837.0,4.0
6183,1ec64Z9X5poSYPT4N1QNm6,89.080,7.0,1.0,-3.349,0.665,0.653,0.2890,0.1960,0.000000,0.0924,0.725,252526.0,4.0
9225,6I6NX6tjGsxFAsIfGzY9lJ,174.063,9.0,1.0,-11.378,0.657,0.421,0.4120,0.3520,0.000218,0.1870,0.181,264747.0,4.0


Mean                                                                      \
   key      mode  loudness danceability    energy speechiness acousticness   
0  7.0  0.833333 -5.760333     0.647333  0.655167    0.216067       0.2571   

  Median                ... Standard deviation                           \
     key mode loudness  ...             energy speechiness acousticness   
0    7.0  1.0  -4.3635  ...           0.158421    0.131639     0.244839   

  Variance                                                          \
       key      mode   loudness danceability    energy speechiness   
0      1.2  0.166667  10.290011      0.01698  0.025097    0.017329   

                
  acousticness  
0     0.059946  

[1 rows x 28 columns]

In [ ]:
import pickle

playlist_list_path = os.path.join(dir_data_processed, "playlists" + str(playlist_count) + ".pkl")

with open(playlist_list_path, "rb") as fin:
    playlist_data = pickle.load(fin)

playlist_frame
playlist_frame = pd.DataFrame(np.arrange())